In [11]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc, html, dash_table as dt
import plotly.express as px
import base64

# Configure OS routines
import os

# Configure the plotting routines
import pandas as pd

#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from CRUD import MongoDB_CRUD

###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name

username = "aacuser"
password = "SNHU1234"
# instantiate the AnimalShelter object
shelter = MongoDB_CRUD(username, password)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned

# Connect to database via CRUD Module
db = MongoDB_CRUD(username, password)

df = pd.DataFrame.from_records(shelter.read_all('AAC', 'animals'))


# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invalid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will return a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'], inplace=True)

#########################
# Dashboard Layout / View
#########################
app = JupyterDash('tbd-JS')

# Load and encode the logo image
image_filename = 'logo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

# Define the layout
app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.A([
            html.Img(
                src='data:image/png;base64,{}'.format(encoded_image.decode()),
                style={
                    'height' : '25%',
                    'width' : '25%',
                    'float' : 'right',
                    'position' : 'relative',
                    'padding-top' : 0,
                    'padding-right' : 0
                })
    ], href='https://unsplash.com/photos/brown-tabby-kitten-sitting-on-floor-nKC772R_qog'),
    html.Br(),
    html.Hr(),
    # Dashboard header
    html.Center(html.B(html.H1('Jose S - Animal Shelter Dashboard'))),
    html.Hr(),
    # Dropdown for interactive filtering
    dcc.Dropdown(
        id='filter-type',
        options=[
            {'label': 'All', 'value': 'All'},
            {'label': 'Water Rescue', 'value': 'WTR'},
            {'label': 'Mountain or Wilderness Rescue', 'value': 'MWR'},
            {'label': 'Disaster/Individual Tracking', 'value': 'DIT'}
        ],
        value='All'
    ),
    # Add interactive filtering options here
    # For example: Radio buttons, drop-downs, checkboxes, etc.
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        # Set up the features for the interactive data table to make it user-friendly for the client
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable=False,
        row_deletable=False,
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current=0,
        page_size=10,
    ),
    html.Br(),
    html.Hr(),
    # This sets up the dashboard so that the chart and the user's geolocation chart are side-by-side",
    html.Div(className='row',
             style={'display': 'flex'},
             children=[
                 html.Div(
                     id='graph-id',
                     className='col s12 m6',
                 ),
                 html.Div(
                     id='map-id',
                     className='col s12 m6',
                 ),
             ]),
    html.Br(),
    html.Hr(),
    # Footer
    html.Footer("Jose Soqui     Programmer/Dev")
])

#############################################
# Interaction Between Components / Controller
#############################################

@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('filter-type', 'value')])
def update_dashboard(value):
    if (value == 'All'):
        df = pd.DataFrame.from_records(shelter.read_all('AAC'))
    elif (value == 'WTR'):
        df = pd.DataFrame.from_records(shelter.read_all('AAC', {"breed" : {"$in": ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]}, "sex_upon_outcome" : "Intact Female", "age_upon_outcome_in_weeks": {"$lte": 156, "$gte": 26}}))
    elif (value == 'MWR'):
        df = pd.DataFrame.from_records(shelter.read_all('AAC', {"breed" : {"$in": ["German Shephard", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]}, "sex_upon_outcome" : "Intact Male", "age_upon_outcome_in_weeks": {"$lte": 156, "$gte": 26}}))
    elif (value == 'DIT'):
        df = pd.DataFrame.from_records(shelter.read_all('AAC', {"breed" : {"$in": ["Doberman Pinscher", "German Shephard", "Golden Retriever", "Bloodhound", "Rottweiler"]}, "sex_upon_outcome" : "Intact Male", "age_upon_outcome_in_weeks": {"$lte": 300, "$gte": 20}}))

    columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
    data=df.to_dict('records')

    return (data,columns)

# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
    dff = pd.DataFrame.from_dict(viewData)
    figure = px.pie(dff, values='age_upon_outcome_in_weeks', names='breed', title='Total Age of Particular Breeds')
    figure.update_traces(textposition='inside')
    figure.update_layout(uniformtext_minsize=12, uniformtext_mode='hide')

    return [
        dcc.Graph(
            id='graph_id',
            figure=figure
        )
    ]

# This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': {'column_id': i},
        'background_color': '#D2F3FF'
    } for i in selected_columns]

# This callback will update the geo-location chart for the selected data entry
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):  
    if viewData is None or index is None:
        return []
    
    dff = pd.DataFrame.from_dict(viewData)
    if 'latitude' not in dff.columns or 'longitude' not in dff.columns:
        return []
    
    row_index = index[0] if index else 0
    
    latitude = dff.iloc[row_index]['latitude']
    longitude = dff.iloc[row_index]['longitude']
    
    if pd.isnull(latitude) or pd.isnull(longitude):
        return []
    
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[latitude, longitude], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            dl.Marker(position=[latitude, longitude], children=[
                dl.Tooltip(dff.iloc[row_index]['tooltip']),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row_index]['name'])
                ])
            ])
        ])
    ]

app.run_server(debug=True)


Dash app running on http://127.0.0.1:27731/
Error in read_all operation: name must be an instance of str
Error in read_all operation: name must be an instance of str
